In [1]:
import os
import warnings
import cv2
import numpy as np
#import tflite_runtime.interpreter as tflite
import tensorflow as tf
import csv
import time
import timeit
import sys
warnings.filterwarnings('ignore')
cwd = os.getcwd()


#gs = "videotestsrc device=/dev/video2 video/x-raw, width=640, height=480, framerate=10/1, format=(string)UYVY ! decodebin ! videoconvert ! appsink"
#cap = cv2.VideoCapture(gs, cv2.CAP_GSTREAMER)
#cap = cv2.VideoCapture("v4l2src device=/dev/video2 ! video/x-raw, width=640, height=480 ! videoconvert ! video/x-raw,format=BGR ! appsink")
cap = cv2.VideoCapture("/dev/video0", cv2.CAP_V4L)
#cap = cv2.VideoCapture(-1)

MODEL_PATH = './model2apis.tflite'
MODEL_NAME = 'model2apis'
DETECTION_THRESHOLD = 0.05
OUTPUT_PATH = './frames1/output{:03}.png'

interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
#interpreter = tflite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()
signature_fn = interpreter.get_signature_runner()
detection_result_image, cnt = [], 0
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

#image_read = cv2.imread
image_resize = cv2.resize
expand_dims = np.expand_dims
uint8 = np.uint8
typecast = np.ndarray.astype
sqee = np.squeeze
draw_rect = cv2.rectangle
draw_text = cv2.putText
fontt = cv2.FONT_HERSHEY_SIMPLEX
#append_bees = bees_per_frame.append
save_image = cv2.imwrite

if cap.isOpened():
    cv2.namedWindow("demo", cv2.WINDOW_AUTOSIZE)
    ctr=0
    while True:
        ret_val, img = cap.read()
        start=time.time()
        if True:
            img = img.astype(uint8)
            resized_img = expand_dims(image_resize(img, (input_height, input_width), interpolation=cv2.INTER_LINEAR), axis=0)

            output = signature_fn(images=resized_img)
            count = int(sqee(output['output_0']))
            scores = sqee(output['output_1'])
            boxes = sqee(output['output_3'])
            results = [{'bounding_box': boxes[i], 'score': scores[i]} for i in range(count) if scores[i] >= DETECTION_THRESHOLD]

            for obj in results:
                ymin, xmin, ymax, xmax = obj['bounding_box']
                xmin = int(xmin * img.shape[1])
                xmax = int(xmax * img.shape[1])
                ymin = int(ymin * img.shape[0])
                ymax = int(ymax * img.shape[0])

                draw_rect(img, (xmin, ymin), (xmax, ymax), (255,0,0), 1)
                y = ymin - 15 if ymin - 15 > 15 else ymin + 15
                label = "{}: {:.0f}%".format('APIS', obj['score'] * 100)
                draw_text(img,label, (xmin, y), fontt, 0.5, (0,0,0), 1)

            #save_image(OUTPUT_PATH.format(ctr//60), img)
            cv2.imshow('demo',img)
        end=time.time()
        print(end-start)
        #print(np.shape(img))
        
        #if ctr%60==0:
        #    cv2.imwrite("./frames/img{:02}.jpg".format(ctr//60),img)
        #width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
        #height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
        # or
        #ctr+=1
        # it gives me 0.0 :/            
        fps = cap.get(cv2.CAP_PROP_FPS)
        print(fps)
        ctr+=1
        cv2.waitKey(1)
else:
    print("camera open failed")
cap.release()
cv2.destroyAllWindows()




2023-04-24 17:54:54.652186: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-24 17:54:54.678803: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-24 17:54:54.679210: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 17:54:55.164827: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


0.04114532470703125
60.0
0.0412898063659668
60.0
0.04065418243408203
60.0
0.03235626220703125
60.0
0.0323026180267334
60.0
0.035152435302734375
60.0
0.03491830825805664
60.0
0.030727624893188477
60.0
0.030460834503173828
60.0
0.03089427947998047
60.0
0.030900001525878906
60.0
0.031037569046020508
60.0
0.03143310546875
60.0
0.031693220138549805
60.0
0.031510353088378906
60.0
0.03661489486694336
60.0
0.03024435043334961
60.0
0.029987812042236328
60.0
0.0299375057220459
60.0
0.029850006103515625
60.0
0.029961585998535156
60.0
0.029645442962646484
60.0
0.030491352081298828
60.0
0.02994823455810547
60.0
0.029390573501586914
60.0
0.02930617332458496
60.0
0.029155492782592773
60.0
0.02928757667541504
60.0
0.029338836669921875
60.0
0.030255556106567383
60.0
0.029243946075439453
60.0
0.029431581497192383
60.0
0.029341697692871094
60.0
0.029631853103637695
60.0
0.029250621795654297
60.0
0.029068470001220703
60.0
0.03045201301574707
60.0
0.029436349868774414
60.0
0.029247760772705078
60.0
0.02923

Only C and default locale supported with the posix collation implementation
Only C and default locale supported with the posix collation implementation
Case insensitive sorting unsupported in the posix collation implementation
Numeric mode unsupported in the posix collation implementation


0.03182101249694824
60.0
0.03193497657775879
60.0
0.036383628845214844
60.0
0.031046390533447266
60.0
0.030843734741210938
60.0
0.03094029426574707
60.0
0.03201627731323242
60.0
0.0307767391204834
60.0
0.031248092651367188
60.0
0.030889511108398438
60.0
0.030291318893432617
60.0
0.03132987022399902
60.0
0.03251290321350098
60.0
0.031569719314575195
60.0
0.031142473220825195
60.0
0.03143811225891113
60.0
0.030150175094604492
60.0
0.030679941177368164
60.0
0.03109002113342285
60.0
0.031247854232788086
60.0
0.030075788497924805
60.0
0.029839038848876953
60.0
0.029657840728759766
60.0
0.030032873153686523
60.0
0.03005361557006836
60.0
0.031319618225097656
60.0
0.030341386795043945
60.0
0.03000044822692871
60.0
0.0301821231842041
60.0
0.0323638916015625
60.0
0.03173041343688965
60.0
0.03068995475769043
60.0
0.031215906143188477
60.0
0.03106975555419922
60.0
0.030844926834106445
60.0
0.03085494041442871
60.0
0.030910968780517578
60.0
0.03127908706665039
60.0
0.030719995498657227
60.0
0.03097

KeyboardInterrupt: 